In [1]:
from calendar import TUESDAY, JANUARY, FEBRUARY, APRIL, JULY, OCTOBER
from datetime import date, timedelta
from typing import List

from lib.schedule import \
    NeverSchedule, \
    DailySchedule, \
    DaySchedule, \
    WeeklySchedule, \
    MonthlySchedule, \
    YearlySchedule, \
    AnySchedule, \
    AllSchedule, \
    ModuloSchedule

START_DATE = date.today()


def print_date_list(dates: List[date]) -> None:
    print('[', end='')
    print(*[item.strftime("%Y-%m-%d : %A") for item in dates], end='', sep='\n ')
    print(']')


# Schedule

A collection of schedule implementations that will take the current date and return if that day
is in the schedule.

## Primitive schedules

The following schedules are the basic building blocks of schedules.

### Never schedule

This is a trivial schedule in that it always returns False


In [2]:
days = [START_DATE + timedelta(days=i) for i in range(1000)]
matches = filter(NeverSchedule().check, days)
print_date_list(matches)

[]


### Daily schedule

This is a trivial schedule in that it always returns True

In [3]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
matches = filter(DailySchedule().check, days)
print_date_list(matches)

[2025-07-31 : Thursday
 2025-08-01 : Friday
 2025-08-02 : Saturday
 2025-08-03 : Sunday
 2025-08-04 : Monday
 2025-08-05 : Tuesday
 2025-08-06 : Wednesday
 2025-08-07 : Thursday
 2025-08-08 : Friday
 2025-08-09 : Saturday]


### Day Schedule

This schedule will only match on the specified day

In [4]:
days = [START_DATE + timedelta(days=i) for i in range(1000)]
matches = filter(DaySchedule(START_DATE + timedelta(days=50)).check, days)
print_date_list(matches)

[2025-09-19 : Friday]


### Weekly schedule

This schedule will match on the specified day of the week

In [5]:
days = [START_DATE + timedelta(days=i) for i in range(50)]
matches = filter(WeeklySchedule(TUESDAY).check, days)
print_date_list(matches)

[2025-08-05 : Tuesday
 2025-08-12 : Tuesday
 2025-08-19 : Tuesday
 2025-08-26 : Tuesday
 2025-09-02 : Tuesday
 2025-09-09 : Tuesday
 2025-09-16 : Tuesday]


### Monthly schedule

This schedule will match on the specified day of the month.

NB. If the current month does not have the specified day (eg. there is no 30th of February in any year)
then the last day of the month will match.

In [6]:
days = [START_DATE + timedelta(days=i) for i in range(500)]
matches = filter(MonthlySchedule(30).check, days)
print_date_list(matches)

[2025-08-30 : Saturday
 2025-09-30 : Tuesday
 2025-10-30 : Thursday
 2025-11-30 : Sunday
 2025-12-30 : Tuesday
 2026-01-30 : Friday
 2026-02-28 : Saturday
 2026-03-30 : Monday
 2026-04-30 : Thursday
 2026-05-30 : Saturday
 2026-06-30 : Tuesday
 2026-07-30 : Thursday
 2026-08-30 : Sunday
 2026-09-30 : Wednesday
 2026-10-30 : Friday
 2026-11-30 : Monday]


### Yearly schedule

This schedule will match on the specified day of specified month.

NB. If the current month does not have the specified day (eg. there is no 30th of February in any year)
then the last day of the month will match.

In [7]:
days = [START_DATE + timedelta(days=i) for i in range(5000)]
matches = filter(YearlySchedule(FEBRUARY, 30).check, days)
print_date_list(matches)

[2026-02-28 : Saturday
 2027-02-28 : Sunday
 2028-02-29 : Tuesday
 2029-02-28 : Wednesday
 2030-02-28 : Thursday
 2031-02-28 : Friday
 2032-02-29 : Sunday
 2033-02-28 : Monday
 2034-02-28 : Tuesday
 2035-02-28 : Wednesday
 2036-02-29 : Friday
 2037-02-28 : Saturday
 2038-02-28 : Sunday
 2039-02-28 : Monday]


## Schedule operators

The following schedules take other schedules and apply an operator to them.

### Any schedule

This represents a boolean `OR` operator for schedules. If any of the child schedules match the current date,
then this schedule will match.

For example, to get a quarterly schedule, you could create four Yearly schedules and Any them together.

In [8]:
days = [START_DATE + timedelta(days=i) for i in range(1000)]
matches = filter(AnySchedule({
    'Every January 1': YearlySchedule(JANUARY, 1),
    'Every April 1': YearlySchedule(APRIL, 1),
    'Every July 1': YearlySchedule(JULY, 1),
    'Every October 1': YearlySchedule(OCTOBER, 1)}).check, days)
print_date_list(matches)

[2025-10-01 : Wednesday
 2026-01-01 : Thursday
 2026-04-01 : Wednesday
 2026-07-01 : Wednesday
 2026-10-01 : Thursday
 2027-01-01 : Friday
 2027-04-01 : Thursday
 2027-07-01 : Thursday
 2027-10-01 : Friday
 2028-01-01 : Saturday
 2028-04-01 : Saturday]


### All schedule

This represents a boolean `AND` operator for schedules. Only if all the child schedules match the current date,
will this schedule match.

In [9]:
days = [START_DATE + timedelta(days=i) for i in range(1000)]
matches = filter(AllSchedule({
    'Every Tuesday': WeeklySchedule(TUESDAY),
    'Every 5th of the month': MonthlySchedule(5)}).check, days)
print_date_list(matches)

[2025-08-05 : Tuesday
 2026-05-05 : Tuesday
 2027-01-05 : Tuesday
 2027-10-05 : Tuesday]


### Modulo schedule

This represents a `MODULO` operator for schedules.

The `start_date` specifies the first date on or after which a match from the child schedule will
result in the Modulo matching.

The `modulo` specifies the number of child matches after each Modulo match before the Modulo will
match again.

For example, to get a schedule for every 4 years, you could use a YearlySchedule for the required date.
Then, apply a ModuloSchedule starting on the first of January and matching every fourth child match.

NB. A ModuloSchedule will assert if a date before the `start_date` is checked

In [10]:
days = [START_DATE + timedelta(days=i) for i in range(5000)]
matches = filter(ModuloSchedule(YearlySchedule(APRIL, 15),
                                start_date=date(START_DATE.year, JANUARY, 1),
                                modulo=4).check, days)
print_date_list(matches)

[2029-04-15 : Sunday
 2033-04-15 : Friday
 2037-04-15 : Wednesday]


## Implementation notes

Schedule checks should be pure functions. That is, any call with the same input should return the
same result.

For most of the implementations, this is true as they are stateless.

However, for the ModuloSchedule to work efficiently, it does implement a cache of dates that have been
checked. Even so, the ModuloSchedule check will always return the same value for the same input as long
as the child schedule check is also a pure function.